<a href="https://colab.research.google.com/github/xuanyu410/114-1PL-Repo/blob/main/%E7%A8%8B%E5%BC%8F%E8%AA%9E%E8%A8%80%E4%BD%9C%E6%A5%AD%E4%B8%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##個人記帳
1.可自行加入日期時間或按enter輸入現在時間。
2.輸入品項後可選擇屬於哪個種類，用於分析圖表。
3.可計算出當日總額即當月總額。

In [60]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)


In [61]:
import pandas as pd
# read data and put it in a dataframe
# 在 google 工作表載入 gsheets
gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/14qT33yoGpMBe868XI6yvd8Hy7DldM4RhovGYNRtCUMk/edit?hl=zh-tw&gid=0#gid=0')

In [68]:
worksheet = gsheets.worksheet("工作表1")
headers = ["日期", "時間", "品項", "種類", "金額"]
first_row = worksheet.row_values(1)
if not first_row or first_row != headers:
    worksheet.clear()
    worksheet.append_row(headers)

In [69]:
 from datetime import datetime
 date_str = input("請輸入日期 (格式：YYYY-MM-DD, Enter=今天): ")
if date_str.strip() == "":
    date_str = datetime.now().strftime("%Y-%m-%d")
else:
    datetime.strptime(date_str, "%Y-%m-%d")

# 時間
time_str = input("請輸入時間 (格式：HH:MM, Enter=現在時間): ")
if time_str.strip() == "":
    time_str = datetime.now().strftime("%H:%M")
else:
    datetime.strptime(time_str, "%H:%M")

# 品項
item = input("請輸入品項: ")

# 金額
amount = float(input("請輸入金額: "))

# 類別選單
categories = ["早餐", "午餐", "晚餐", "點心", "飲料", "衣服", "娛樂", "交通"]
print("請選擇種類：")
for i, c in enumerate(categories, 1):
    print(f"{i}. {c}")
choice = int(input("輸入數字選擇: "))
category = categories[choice - 1]
worksheet.append_row([date_str, time_str, item, category, amount], value_input_option="USER_ENTERED")
print("✅ 新增成功！")

請輸入日期 (格式：YYYY-MM-DD, Enter=今天): 
請輸入時間 (格式：HH:MM, Enter=現在時間): 
請輸入品項: 麵包
請輸入金額: 57
請選擇種類：
1. 早餐
2. 午餐
3. 晚餐
4. 點心
5. 飲料
6. 衣服
7. 娛樂
8. 交通
輸入數字選擇: 1
✅ 新增成功！


In [70]:
sheets = worksheet.get_all_values()
df = pd.DataFrame(sheets[1:], columns=sheets[0])
df["金額"] = df["金額"].replace(r"[^\d.]+", "", regex=True).astype(float)
df["日期_dt"] = pd.to_datetime(df["日期"], errors="coerce")
df["日期_str"] = df["日期_dt"].dt.strftime("%Y-%m-%d")


In [71]:
today_str = datetime.now().strftime("%Y-%m-%d")  # 或使用你輸入的 date_str
today_total = df[df["日期_str"] == today_str]["金額"].astype(float).sum()

In [72]:
# 找當天的所有列
rows = [i+2 for i, v in enumerate(df["日期"]) if v == today]  # +2 因為 worksheet row 從1開始且有標題

# 在最後一行新增「當天總支出」
worksheet.append_row([today_str, "", "當天總支出", "", today_total], value_input_option="USER_ENTERED")
print(f"📌 {today_str} 當天總支出已更新：{today_total} 元")


📌 2025-09-11 當天總支出已更新：92.0 元


In [73]:
year = int(date_str[:4])
month = int(date_str[5:7])
month_total = df[(df["日期_dt"].dt.year == year) & (df["日期_dt"].dt.month == month)]["金額"].sum()
print(f"📌 {date_str[:7]} 當月總支出：{month_total} 元")

📌 2025-09 當月總支出：92.0 元
